In [31]:
import subprocess
from faster_whisper import WhisperModel
# from lista import segmentos_lista
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
import os
import asyncio
import json
import ast
from langchain_core.output_parsers import JsonOutputParser
# from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
from langchain_together import ChatTogether
from TTS.api import TTS
import torch
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts
import numpy as np
from pydub import AudioSegment
from tqdm import tqdm

In [32]:
load_dotenv()

os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
os.environ['TOGETHER_API_KEY'] = os.getenv('TOGETHER_API_KEY')

In [33]:
enlace_video = "https://www.youtube.com/watch?v=b_ANHgTHNSI"
audio_salida = './src/media/audio.wav'
video_salida = './src/media/video.webm'
tamanio_modelo = "medium"
dispositivo = "cuda"
tipo_calculo = "float16"
audio_final_salida = './src/media/audio_final.wav'
video_final_salida = './src/media/video_final.mp4'

In [34]:
model = WhisperModel(tamanio_modelo, device=dispositivo,
                         compute_type=tipo_calculo)
segmentos_generador, info = model.transcribe(
        audio_salida, task="transcribe", language="en", beam_size=5)
segmentos_lista = list(segmentos_generador)

### LISTA DE SEEGMENTOS MODIFICADOS

In [35]:
lista_provisional = []

for segmento in segmentos_lista:
    segmento_dict = {
        'id': segmento.id,
        'start': segmento.start,
        'end': segmento.end,
        'text': segmento.text,
        'start_transcripcion': None,
        'end_transcripcion': None
    }
    lista_provisional.append(segmento_dict)
    print(segmento)

Segment(id=1, seek=2714, start=0.0, end=3.4, text=" let's talk about tech stack. I get a lot of suggestion from software engineers who", tokens=[50364, 718, 311, 751, 466, 7553, 8630, 13, 286, 483, 257, 688, 295, 16541, 490, 4722, 11955, 567, 50534], temperature=0.0, avg_logprob=-0.13347868217054262, compression_ratio=1.7003058103975535, no_speech_prob=0.0643310546875, words=None)
Segment(id=2, seek=2714, start=3.4, end=6.72, text=' mentioned that I could save some money by switching to another tech stack or', tokens=[50534, 2835, 300, 286, 727, 3155, 512, 1460, 538, 16493, 281, 1071, 7553, 8630, 420, 50700], temperature=0.0, avg_logprob=-0.13347868217054262, compression_ratio=1.7003058103975535, no_speech_prob=0.0643310546875, words=None)
Segment(id=3, seek=2714, start=6.72, end=10.84, text=" maybe a new UI components library that's trendy but I never listened to them and I", tokens=[50700, 1310, 257, 777, 15682, 6677, 6405, 300, 311, 38596, 457, 286, 1128, 13207, 281, 552, 293, 286, 

# CHATBOT FORMATEANDO SEGMENTOS EN INGLES

In [36]:
llm = ChatTogether(model="meta-llama/Llama-3-70b-chat-hf", temperature=0)

In [37]:
lista_tamanio_original = len(lista_provisional)

lista_traduccion = []
lista_ciclica = []
chunk_size = 10
iteracion = 0

In [38]:
while len(lista_provisional) > 0:
        lista_200 = lista_provisional[:chunk_size]
        # creamos una lista de una cadena de 20 elementos
        cadena_200 = json.dumps(lista_200, ensure_ascii=True)

        template = """
Eres un experto traductor del inglés al español y un excelente programador.

# TAREA:
Tu objetivo es tomar el JSON que te dan, que es una lista de las transcripciones de un video con el id, start, end y el text, y traducir el text al español :

1. Mantén el formato de JSON, sustituyendo únicamente las cadenas de cada text por la traducción al español.
3. Traduce el texto al español, pero conserva las marcas o terminologías académicas como clases de programación en inglés.
4. Asegúrate de que las frases sean coherentes. Si no tienen sentido al traducirlas literalmente, ajústalas para que mantengan el contexto.
5. Mantén la misma cantidad de items en la representación de la lista.
7. Entregarás tu respuesta en fomato JSON donde la key se llama 'traduccion' y el valor sea la lista de strings.

# CONTEXTO:
Estás trabajando para una empresa que transcribe vídeos del inglés al español. A tí te envian un JSON de python para que tu traduzcas cada parametro text al español. Tu tarea es encargarte de traducir cada text y de entregar un JSON respetando el formato con el que te lo entregan donde entregarás un JSON con una key 'traduccion' y como valor, la lista que te pasaron traducida más sus parametros. Es muy importante que repetes el formato .

# Ejemplo 1
input:
[
    {{"id": 1, "start": 0.0, "end": 4.0, "text": "people will generate workbook to learn a new language I use GPT-4 under the hood"}},
    {{"id": 2, "start": 4.0, "end": 8.0, "text": "and I spend about $20 per month in open AI credit if I need any other cloud"}},
    {{"id": 3, "start": 8.0, "end": 12.0, "text": "services I would go to Amazon Web Services the two most common that I use"}},
    {{"id": 4, "start": 12.0, "end": 16.0, "text": "are AWS S3 a big bucket where I can host images this is what I use on indie page"}},
    {{"id": 5, "start": 16.0, "end": 20.0, "text": "where people upload pictures of their startups or their profile picture this"}},
    {{"id": 6, "start": 20.0, "end": 24.0, "text": "is where the image would go and I also use from time to time AWS in cloud front"}},
    {{"id": 7, "start": 24.0, "end": 28.0, "text": "it's a CDN so images or videos load faster for things like a course for"}},
    {{"id": 8, "start": 28.0, "end": 32.0, "text": "instance and a final touch it is not really part of a tech stack but I hire"}},
    {{"id": 9, "start": 32.0, "end": 36.0, "text": "a junior software engineer called github copilot it cost me $10 per month and"}},
    {{"id": 10, "start": 36.0, "end": 40.0, "text": "it's helping me code way faster I'll link to another video I made about how I"}}
]

output:
{{
    "traduccion": [
        {{"id": 1, "start": 0.0, "end": 4.0, "text": "la gente generará un libro de trabajo para aprender un nuevo idioma yo uso GPT-4 bajo el capó"}},
        {{"id": 2, "start": 4.0, "end": 8.0, "text": "y gasto unos 20 dólares al mes en crédito abierto AI si necesito alguna otra nube"}},
        {{"id": 3, "start": 8.0, "end": 12.0, "text": "servicios que iría a Amazon Web Services los dos más comunes que yo uso"}},
        {{"id": 4, "start": 12.0, "end": 16.0, "text": "son AWS S3 un gran cubo donde puedo alojar imágenes esto es lo que uso en la página indie"}},
        {{"id": 5, "start": 16.0, "end": 20.0, "text": "donde la gente sube fotos de sus startups o su foto de perfil esto"}},
        {{"id": 6, "start": 20.0, "end": 24.0, "text": "es donde la imagen iría y también uso de vez en cuando AWS en cloud front"}},
        {{"id": 7, "start": 24.0, "end": 28.0, "text": "es un CDN para que las imágenes o vídeos carguen más rápido para cosas como un curso para"}},
        {{"id": 8, "start": 28.0, "end": 32.0, "text": "instancia y un toque final no es realmente parte de una pila de tecnología, pero yo contrato"}},
        {{"id": 9, "start": 32.0, "end": 36.0, "text": "un ingeniero de software junior llamado github copilot que me cuesta $10 por mes y"}},
        {{"id": 10, "start": 36.0, "end": 40.0, "text": "me está ayudando a codificar mucho más rápido voy a enlazar a otro video que hice sobre cómo"}}
    ]
}}

# Ejemplo 2
input:
[
    {{"id": 11, "start": 40.0, "end": 44.0, "text": "computation started becoming less expensive, more compute. Then we have the computer that we have"}},
    {{"id": 12, "start": 44.0, "end": 48.0, "text": "today. Andhri Karpathy is arguing, can we have a similar vision for LLM? And where the vision is"}},
    {{"id": 13, "start": 48.0, "end": 52.0, "text": "you keep LLM at the center, right? You keep LLM at the center. And at the center with LLM,"}},
    {{"id": 14, "start": 52.0, "end": 56.0, "text": "you have RAM, which is the short-term memory or the context window. Then you have long-term memory,"}},
    {{"id": 15, "start": 56.0, "end": 60.0, "text": "the disk system that can be used with RAG. Then you have the agent structure that you have with"}},
    {{"id": 16, "start": 60.0, "end": 64.0, "text": "tools. And then you connect it with internet. And when you connect it with other LLMs to have"}},
    {{"id": 17, "start": 64.0, "end": 68.0, "text": "like a multi-agent setup or like a peripheral setup. And then you have your peripheral devices"}},
    {{"id": 18, "start": 68.0, "end": 72.0, "text": "where you have got audio and video. Can we put together a system with all these things working"}},
    {{"id": 19, "start": 72.0, "end": 76.0, "text": "towards a common goal? And that will ideally become your large language model operating system."}},
    {{"id": 20, "start": 76.0, "end": 80.0, "text": "This is quite a vision at this point. There are certain implementations available at this point."}}
]


output:
{{
    "traduccion": [
        {{"id": 11, "start": 40.0, "end": 44.0, "text": "la computación empezó a ser menos cara, más computacional. Entonces tenemos el ordenador que tenemos"}},
        {{"id": 12, "start": 44.0, "end": 48.0, "text": "hoy. Andhri Karpathy argumenta, ¿podemos tener una visión similar para LLM? Y dónde está la visión"}},
        {{"id": 13, "start": 48.0, "end": 52.0, "text": "mantienes el LLM en el centro, ¿verdad? Mantienes el LLM en el centro. Y en el centro con LLM,"}},
        {{"id": 14, "start": 52.0, "end": 56.0, "text": "tienes la RAM, que es la memoria a corto plazo o la ventana contextual. Luego tienes la memoria a largo plazo,"}},
        {{"id": 15, "start": 56.0, "end": 60.0, "text": "el sistema de disco que se puede utilizar con RAG. Luego tienes la estructura de agentes que tienes con"}},
        {{"id": 16, "start": 60.0, "end": 64.0, "text": "herramientas. Y luego lo conectas con Internet. Y cuando lo conectas con otros LLMs para tener"}},
        {{"id": 17, "start": 64.0, "end": 68.0, "text": "como una configuración multi-agente o como una configuración periférica. Y entonces tienes tus dispositivos periféricos"}},
        {{"id": 18, "start": 68.0, "end": 72.0, "text": "donde tienes audio y video. ¿Podemos montar un sistema con todas estas cosas funcionando"}},
        {{"id": 19, "start": 72.0, "end": 76.0, "text": "¿hacia un objetivo común? Y eso se convertirá idealmente en tu gran sistema operativo modelo de lenguaje"}},
        {{"id": 20, "start": 76.0, "end": 80.0, "text": "Esta es toda una visión en este momento. Hay ciertas implementaciones disponibles en este momento"}}
    ]
}}


# Ejemplo 3
input:
[
    {{"id": 21, "start": 80.0, "end": 84.0, "text": "let's talk about tech stack. I get a lot of suggestion from software engineers who"}},
    {{"id": 22, "start": 84.0, "end": 88.0, "text": "mentioned that I could save some money by switching to another tech stack or"}},
    {{"id": 23, "start": 88.0, "end": 92.0, "text": "maybe a new UI components library that's trendy but I never listened to them and I"}},
    {{"id": 24, "start": 92.0, "end": 96.0, "text": "always stick to the exact same tech stack and thanks to that in the last two"}},
    {{"id": 25, "start": 96.0, "end": 100.0, "text": "years I was able to ship 21 products and those products are getting 260,000"}},
    {{"id": 26, "start": 100.0, "end": 104.0, "text": "page views per month and are making over 50,000 US dollars in the revenue and I'm"}},
    {{"id": 27, "start": 104.0, "end": 108.0, "text": "working solo and I handle the customer support and there is barely any problem"}},
    {{"id": 28, "start": 108.0, "end": 112.0, "text": "and the tech stack only cost 67 US dollar per month. Alright now let me show you"}},
    {{"id": 29, "start": 112.0, "end": 116.0, "text": "exactly the tech that I use. Alright so for the front-end part I use React.js"}},
    {{"id": 30, "start": 116.0, "end": 120.0, "text": "it's a very popular JavaScript library it has a bunch of extra library like a"}}
]


output:
{{
    "traduccion": [
        {{"id": 21, "start": 80.0, "end": 84.0, "text": "hablemos de la pila tecnológica. Recibo muchas sugerencias de ingenieros de software que"}},
        {{"id": 22, "start": 84.0, "end": 88.0, "text": "me mencionaron que podría ahorrar dinero cambiando a otra pila tecnológica o"}},
        {{"id": 23, "start": 88.0, "end": 92.0, "text": "tal vez una nueva biblioteca de componentes UI que es trendy, pero nunca las he seguido y yo"}},
        {{"id": 24, "start": 92.0, "end": 96.0, "text": "siempre me mantengo en la misma pila tecnológica y gracias a eso en los últimos dos años"}},
        {{"id": 25, "start": 96.0, "end": 100.0, "text": "he podido lanzar 21 productos y esos productos están recibiendo 260,000"}},
        {{"id": 26, "start": 100.0, "end": 104.0, "text": "vistas por mes y están generando más de 50,000 dólares EE. UU. En ingresos y estoy yo"}},
        {{"id": 27, "start": 104.0, "end": 108.0, "text": "trabajando solo y manejo el soporte al cliente y casi no hay problemas"}},
        {{"id": 28, "start": 108.0, "end": 112.0, "text": "y la pila tecnológica cuesta 67 dólares EE. UU. Al mes. Bueno, ahora les mostraré"}},
        {{"id": 29, "start": 112.0, "end": 116.0, "text": "exactamente la tecnología que uso. Está bien, para la parte del front-end uso React.js"}},
        {{"id": 30, "start": 116.0, "end": 120.0, "text": "es una biblioteca de JavaScript muy popular tiene muchas otras librerías como una"}}
    ]
}}

# NOTA:
- Contesta solo con la respuesta en forma de JSON. No respondas nada que no sea JSON.
- Asegúrate de mantener la estructura de las listas con el valor de la key, entregando solo la lista con los ítems traducidos {{traduccion:[{{item1}}, {{item2}}, {{item3}}, etc.]}}.
- Por favor, proporciona una respuesta en formato de JSON. El JSON debe contener una lista. La lista debe tener exactamente el mismo número de elementos que la lista original que te he dado. No agregues ni elimines ningún elemento. Los elementos tiene que tener exactamente los mismos parametros que el original solo debes cambiar el text con la traduccion.
- Es muy importante que detectes correctamente cada elemento de la lista, el buen funcionamiento de la empresa depende de ello.
- Antes de da tu respuesta compara los ids de los items de la lista original con los de la nueva y asegurate de que esten todos , si falta alguno arreglalo antes de responder.
- Asegurate en la lista del JSON de la respuesta estan los 10 dicts como en el original.

# JSON A TRANSFORMAR:
{cadena_200}

# Respuesta:
{{"traduccion": [{{"id":x, "start":y, "end":z, "text":""}}, {{"id":x, "start":y, "end":z, "text":""}}, {{"id":x, "start":y, "end":z, "text":""}}, {{"id":x, "start":y, "end":z, "text":""}}, {{"id":x, "start":y, "end":z, "text":""}}, {{"id":x, "start":y, "end":z, "text":""}},{{"id":x, "start":y, "end":z, "text":""}}, {{"id":x, "start":y, "end":z, "text":""}}, {{"id":x, "start":y, "end":z, "text":""}}, {{"id":x, "start":y, "end":z, "text":""}}]}}

"""

        prompt_template = PromptTemplate(
            template=template, input_variables=["cadena_200"])

        chain = prompt_template | llm

        respuesta = await chain.ainvoke(input={"cadena_200": cadena_200})

        mensaje = respuesta.content

        # Obtenemos la lista pero aun es str
        lista_string = mensaje[mensaje.find('['):mensaje.rfind(']')+1]

        # Convertimos a lista str en tipo list
        lista_ciclica = ast.literal_eval(lista_string)

        iteracion += 1

        print(
            f"------------------- Iteración {iteracion} -----------------------")
        lista_provisional = lista_provisional[chunk_size:]
        lista_traduccion.extend(lista_ciclica)

------------------- Iteración 1 -----------------------
------------------- Iteración 2 -----------------------
------------------- Iteración 3 -----------------------
------------------- Iteración 4 -----------------------
------------------- Iteración 5 -----------------------
------------------- Iteración 6 -----------------------
------------------- Iteración 7 -----------------------
------------------- Iteración 8 -----------------------


In [ ]:
print(lista_traduccion)
print("*******************************************************************")
print(f"Tamañio lista original = {lista_tamanio_original}")
print(f"Tamaño nueva lista = {len(lista_traduccion)}")

[{'id': 1, 'start': 0.0, 'end': 3.4, 'text': 'hablemos de la pila tecnológica. Recibo muchas sugerencias de ingenieros de software que'}, {'id': 2, 'start': 3.4, 'end': 6.72, 'text': 'me mencionaron que podría ahorrar dinero cambiando a otra pila tecnológica o'}, {'id': 3, 'start': 6.72, 'end': 10.84, 'text': 'tal vez una nueva biblioteca de componentes UI que es trendy, pero nunca las he seguido y yo'}, {'id': 4, 'start': 10.84, 'end': 14.52, 'text': 'siempre me mantengo en la misma pila tecnológica y gracias a eso en los últimos dos'}, {'id': 5, 'start': 14.52, 'end': 19.12, 'text': 'años he podido lanzar 21 productos y esos productos están recibiendo 260,000'}, {'id': 6, 'start': 19.12, 'end': 23.22, 'text': 'vistas por mes y están generando más de 50,000 dólares EE. UU. en ingresos y estoy yo'}, {'id': 7, 'start': 23.22, 'end': 27.14, 'text': 'trabajando solo y manejo el soporte al cliente y casi no hay problemas'}, {'id': 8, 'start': 27.14, 'end': 31.54, 'text': 'y la pila tecnoló

# Segunda Pregunta al LLM

In [ ]:
copia_lista = lista_traduccion.copy()
lista_traduccion2 = []
lista_ciclica2 = []
chunk_size2 = 10
iteracion2 = 0

In [ ]:
while len(copia_lista) > 0:
    lista_200 = copia_lista[:chunk_size2]
    cadena_200 = json.dumps(lista_200, ensure_ascii=True)
    
    template = """
    # ROL:
    Eres un experto en gramática y lingüista que se maneja bien con la programación.
    
    # TAREA:
    Te llegará una lista de dict de una futura transcripcion para un video, tu tarea es fijarte en los valores de los campos de text de cada objeto que tengan coherencia y continuidad entre ellos,para esto haras lo siguiente:
    1- Harás que el global de todos los text tengan continuidad entre ellos sabiendo que el ultimo text seguramente si esta cortado su text continue en otra hoja.
    2- Modificaras todos los valores de los campos text escribiendolos tal y como se pronunciarian, por ejemplo:
    - MongoDB -> Mongo debe
    - LLM -> ele ele eme
    - UI -> Uai
    - HTML -> ache te eme ele
    - EE. UU. -> estados unidos
    3- Todos los valores numéricos tambien los escribirás como se leerian y si van acompañados de la moneda tambien escribirias la moneda , porejemplo:
    - 200$ -> doscientos dolares
    - 5€ o 5 eur -> cinco euros
    - 1.985 -> mil novecientos ocheinta y cinco
    - XV -> Siglo quince
    4- Los terminos en ingles importantes como marcas o clases en programación escribelas tambien como se pronunciarian:
    - nike -> naik
    - python -> paiton
    - wifi -> waifai
    5- Debes mantener la estructura de la lista y cada uno de los dict tal y como te llegaron modificando solo el valor del text si lo consideras necesario.
    6- Tu respuesta debe ser un JSON ya que es el formato que te solicitan, este JSON tendra una lista y la lista estara compuesta de los dict que te pasaron con los cambios en text que hiciste.
    
    # CONTEXTO:
    Estas contratado para escribir foneticamente para que al leer exactamente las palabras escritas sonaran como deberian. Tu empresa se encarga de preparar la lista con los dict de informacion que luego se pasaran a un desarrollador que generará un avatar que leerá los text que tu escribas , es por ello que es muy importante que escribas exactamente como sonarian las frases para que cuando lo lea el avatar suene bien. LLevas 3 años trabajando en esta empresa que es una startap de avance tecnologico en avatares de IA .
    
    # EJEMPLO DE RESPUESTA:
    {{
    "traduccion": [
        {{"id": 1, "start": 0.0, "end": 4.0, "text": "la gente generará un libro de trabajo para aprender un nuevo idioma yo uso GePeTe-cuatro bajo el capó"}},
        {{"id": 2, "start": 4.0, "end": 8.0, "text": "y gasto unos veinte dólares al mes en crédito abierto A I si necesito alguna otra cloude"}},
        {{"id": 3, "start": 8.0, "end": 12.0, "text": "service iría a Amazon Web Services los dos más comunes que yo uso"}},
        {{"id": 4, "start": 12.0, "end": 16.0, "text": "son A uve  eSe eSe tres  un gran cubo donde puedo alojar imágenes esto es lo que uso en la página indi"}},
        {{"id": 5, "start": 16.0, "end": 20.0, "text": "donde la gente sube fotos de sus startaps o su foto de perfil esto"}},
        {{"id": 6, "start": 20.0, "end": 24.0, "text": "es donde la imagen iría y también uso de vez en cuando A uve ese en cloud front"}},
        {{"id": 7, "start": 24.0, "end": 28.0, "text": "es un Ce De eNe para que las imágenes o vídeos carguen más rápido para cosas como un curso para"}},
        {{"id": 8, "start": 28.0, "end": 32.0, "text": "instancia y un toque final no es realmente parte de una pila de tecnología, pero yo contrato"}},
        {{"id": 9, "start": 32.0, "end": 36.0, "text": "un ingeniero de software junior llamado guit jub copilot que me cuesta diez dolares por mes y"}},
        {{"id": 10, "start": 36.0, "end": 40.0, "text": "me está ayudando a codificar mucho más rápido, voy a enlazar a otro video que hice sobre cómo"}}
    ]
}}

# NOTA:
- Asegurate de mantener el formato de salida {{traduccion:[{{item1}}, {{item2}}, {{item3}}, etc.]}}
- Ajusta los textos para que tengan coherencia en el global.
- Escribe los textos como se pronunciarian.

# JSON A TRANSFORMAR:
{cadena_200}

# Respuesta:
{{"traduccion": [{{"id":x, "start":y, "end":z, "text":""}}, {{"id":x, "start":y, "end":z, "text":""}}, {{"id":x, "start":y, "end":z, "text":""}}, {{"id":x, "start":y, "end":z, "text":""}}, {{"id":x, "start":y, "end":z, "text":""}}, {{"id":x, "start":y, "end":z, "text":""}},{{"id":x, "start":y, "end":z, "text":""}}, {{"id":x, "start":y, "end":z, "text":""}}, {{"id":x, "start":y, "end":z, "text":""}}, {{"id":x, "start":y, "end":z, "text":""}}]}}
    """
    
    
    prompt_template = PromptTemplate(template=template, input_variables=["cadena_200"])

    chain = prompt_template | llm

    respuesta = await chain.ainvoke(input={"cadena_200": cadena_200})

    mensaje = respuesta.content
    print(mensaje)
    
    print("Iteración ")
    
    
    
    
    
    
    copia_lista = copia_lista[chunk_size2:]